In [155]:
import pandas as pd
import numpy as np

In [156]:
# 샘플데이터
import seaborn as sns

# 타이타닉 데이터 로드
df_titanic = sns.load_dataset("titanic").dropna()
df_titanic = df_titanic.loc[:,['survived','pclass','sex','age','sibsp','parch','embarked']]

# 상위 5개 행 출력
df_titanic.head()



survived  pclass     sex   age  sibsp  parch embarked
1          1       1  female  38.0      1      0        C
3          1       1  female  35.0      1      0        S
6          0       1    male  54.0      0      0        S
10         1       3  female   4.0      1      1        S
11         1       1  female  58.0      0      0        S

#### 파생컬럼 생성
* `컬럼간 연산`
* `컬럼.map()` : 값 매핑
* `컬럼.qcut()` : 분위수 기반 구간화
* `컬럼.rank()` : 순위 추출
* `pd.cut()` : 구간 분할
* `np.where()` : 조건 분기 처리
* `np.select()` : 조건 분기 처리
* `컬럼.apply()` : 함수 적용

#### `컬럼.map()`
* `컬럼.map(딕셔너리)`
* `컬럼.map(함수)`
* `컬럼.map(시리즈)`

In [158]:
# pclass 컬럼을 이용하여 class 컬럼 추가
# 1 -> 'first'
# 2 -> 'second'
# 3 -> 'thrid'

df_titanic['class'] = df_titanic['pclass'].map({1:'first',
                                                2:'second',
                                                3:'third'})

df_titanic.head()


survived  pclass     sex   age  sibsp  parch embarked  alone  class
1          1       1  female  38.0      1      0        C  False  first
3          1       1  female  35.0      1      0        S  False  first
6          0       1    male  54.0      0      0        S   True  first
10         1       3  female   4.0      1      1        S  False  third
11         1       1  female  58.0      0      0        S   True  first

In [159]:
# embarked 컬럼을 이용하여 embark_town 컬럼 추가
# 'C' -> 'Cherbourg'
# 'Q' -> 'Queenstown'
# 'S' -> 'Southampton'

df_titanic['embark_town'] = df_titanic['embarked'].map({'C':'Cherbourg',
                                                        'Q':'Queenstown',
                                                        'S':'Southampton'})
df_titanic.sample(5)

survived  pclass     sex   age  sibsp  parch embarked  alone  class  \
544         0       1    male  50.0      1      0        C  False  first   
763         1       1  female  36.0      1      2        S  False  first   
275         1       1  female  63.0      1      0        S  False  first   
632         1       1    male  32.0      0      0        C   True  first   
3           1       1  female  35.0      1      0        S  False  first   

     embark_town  
544    Cherbourg  
763  Southampton  
275  Southampton  
632    Cherbourg  
3    Southampton

In [160]:
# survived 컬럼 이용하여 alive 컬럼 생성
# 1 --> 'yes'
# 0 --> 'no'

df_titanic['alive'] = df_titanic['survived'].map({0:'no', 1:'yes'})
df_titanic.sample(5)

survived  pclass     sex   age  sibsp  parch embarked  alone  class  \
337         1       1  female  41.0      0      0        C   True  first   
698         0       1    male  49.0      1      1        C  False  first   
309         1       1  female  30.0      0      0        C   True  first   
124         0       1    male  54.0      0      1        S  False  first   
707         1       1    male  42.0      0      0        S   True  first   

     embark_town alive  
337    Cherbourg   yes  
698    Cherbourg    no  
309    Cherbourg   yes  
124  Southampton    no  
707  Southampton   yes

#### `np.where()`
* `np.where(조건, 참일때값, 거짓일때값)`

In [154]:
cond1 = df_titanic['sex']=='male'
cond2 = df_titanic['age']>=20

df_titanic['male_adult'] = np.where(cond1&cond2, 'yes', 'no')
df_titanic.sample(5)

survived  pclass     sex   age  sibsp  parch embarked  alone  class  \
377         0       1    male  27.0      0      2        C  False  first   
512         1       1    male  36.0      0      0        S   True  first   
867         0       1    male  31.0      0      0        S   True  first   
871         1       1  female  47.0      1      1        S  False  first   
75          0       3    male  25.0      0      0        S   True  third   

     embark_town alive male_adult  
377    Cherbourg    no        yes  
512  Southampton   yes        yes  
867  Southampton    no        yes  
871  Southampton   yes         no  
75   Southampton    no        yes

#### `np.select()`
* `np.select(조건목록, 각조건에매핑할값, 디폴트)`

In [161]:
##################################################################
# who  컬럼 추가
#   man(20세 이상 남자), woman(20세 이상 여자), child(20세 미만)
##################################################################

# 조건 목록
cond_man = (df_titanic['sex']=='male') & (df_titanic['age'] >=20)
cond_woman = (df_titanic['sex']=='female') & (df_titanic['age'] >=20)
conditions = [cond_man, cond_woman]

# 각 조건에 매핑할 값
choices = ['man', 'woman']

df_titanic['who'] = np.select(conditions, choices, default='child')
df_titanic.sample(5)

survived  pclass     sex   age  sibsp  parch embarked  alone   class  \
587         1       1    male  60.0      1      1        C  False   first   
504         1       1  female  16.0      0      0        S   True   first   
486         1       1  female  35.0      1      0        S  False   first   
681         1       1    male  27.0      0      0        C   True   first   
66          1       2  female  29.0      0      0        S   True  second   

     embark_town alive    who  
587    Cherbourg   yes    man  
504  Southampton   yes  child  
486  Southampton   yes  woman  
681    Cherbourg   yes    man  
66   Southampton   yes  woman

#### `pd.cut()`
* `pd.cut(컬럼, 구간)`
* 주요 파라미터
    * `labels` : 각 구간에 부여할 라벨
    * `right` : 오른쪽 경계 포함 여부

In [163]:
##################################################################
# 연령대 컬럼 추가
#   10세미만, 10대, 20대, ..., 80세 이상
##################################################################

bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 150]
labels = ['10세 미만', '10대', '20대', '30대', '40대', '50대', '60대', '70대', '80세 이상']

df_titanic['age_group'] = pd.cut(df_titanic['age'], bins=bins, 
                                labels=labels, 
                                right=False)

df_titanic.sample(5)

survived  pclass     sex   age  sibsp  parch embarked  alone  class  \
857         1       1    male  51.0      0      0        S   True  first   
369         1       1  female  24.0      0      0        C   True  first   
591         1       1  female  52.0      1      0        C  False  first   
251         0       3  female  29.0      1      1        S  False  third   
879         1       1  female  56.0      0      1        C  False  first   

     embark_town alive    who age_group  
857  Southampton   yes    man       50대  
369    Cherbourg   yes  woman       20대  
591    Cherbourg   yes  woman       50대  
251  Southampton    no  woman       20대  
879    Cherbourg   yes  woman       50대

In [198]:
# 샘플 데이터
df_scores = pd.read_csv('data/scores.csv').fillna(0)
df_scores['평균'] = ((df_scores['국어']+df_scores['영어']+df_scores['수학'])/3).round(2)
df_scores

학번      이름 성별     국어   영어     수학      평균
0    1    kate  f  100.0   90   95.0   95.00
1    2  olivia  f   90.0   80   75.0   81.67
2    3    emma  f   95.0  100  100.0   98.33
3    4  sophia  f  100.0  100  100.0  100.00
4    5     amy  f    0.0   35   60.0   31.67
5    6   james  m   70.0   75   65.0   70.00
6    7  daniel  m   80.0   90   55.0   75.00
7    8   peter  m   50.0   60    0.0   36.67
8    9   kevin  m  100.0  100   90.0   96.67
9   10    jack  m   90.0   95   70.0   85.00
10  11   james  m   70.0   75   65.0   70.00

#### `컬럼.rank()`
>* 주요 파라미터
>* method
>    * `'average'`: 동점인 경우 평균순위 부여
>    * `'min'` : 동점인 경우 최소 순위 부여
>    * `'max'` : 동점인 경우 최대 순위 부여
>    * `'first'` : 데이터 등장 순서대로 부여
>    * `'dense'` : min과 유사하지만 순위 간격 없이 연속적으로 부여
>* ascending
>    * `True` : 오름차순 순위
>    * `False` : 내림차순 순위

In [199]:
df_scores['순위'] = df_scores['평균'].rank(ascending=False,
                                       method='min').astype('int')
df_scores

학번      이름 성별     국어   영어     수학      평균  순위
0    1    kate  f  100.0   90   95.0   95.00   4
1    2  olivia  f   90.0   80   75.0   81.67   6
2    3    emma  f   95.0  100  100.0   98.33   2
3    4  sophia  f  100.0  100  100.0  100.00   1
4    5     amy  f    0.0   35   60.0   31.67  11
5    6   james  m   70.0   75   65.0   70.00   8
6    7  daniel  m   80.0   90   55.0   75.00   7
7    8   peter  m   50.0   60    0.0   36.67  10
8    9   kevin  m  100.0  100   90.0   96.67   3
9   10    jack  m   90.0   95   70.0   85.00   5
10  11   james  m   70.0   75   65.0   70.00   8

#### `pd.qcut()`
* `pd.qcut(컬럼, 구간수)`
* `pd.qcut(컬럼, 구간리스트)`
>* 추가옵션 
>    * labels : 구간이름

In [200]:
labels = ['Low', 'Middle', 'High']
df_scores['grade'] = pd.qcut(df_scores['평균'], 3, labels=labels)
df_scores

학번      이름 성별     국어   영어     수학      평균  순위   grade
0    1    kate  f  100.0   90   95.0   95.00   4    High
1    2  olivia  f   90.0   80   75.0   81.67   6  Middle
2    3    emma  f   95.0  100  100.0   98.33   2    High
3    4  sophia  f  100.0  100  100.0  100.00   1    High
4    5     amy  f    0.0   35   60.0   31.67  11     Low
5    6   james  m   70.0   75   65.0   70.00   8     Low
6    7  daniel  m   80.0   90   55.0   75.00   7  Middle
7    8   peter  m   50.0   60    0.0   36.67  10     Low
8    9   kevin  m  100.0  100   90.0   96.67   3    High
9   10    jack  m   90.0   95   70.0   85.00   5  Middle
10  11   james  m   70.0   75   65.0   70.00   8     Low

#### `컬럼.apply()`
* `컬럼.apply(함수, 매개변수명=매개변수값)`

In [204]:
# 모든 학생에게 점수 추가 부여(100점을 넘을 수 없음)
def add_score(score, plus_score):
    result = score + plus_score
    if result>100 : result = 100
    return result

df_scores['영어+10'] = df_scores['영어'].apply(add_score, plus_score=10)
display(df_scores)

학번      이름 성별     국어   영어     수학      평균  순위   grade  영어+10
0    1    kate  f  100.0  100   95.0   95.00   4    High    100
1    2  olivia  f   90.0   90   75.0   81.67   6  Middle    100
2    3    emma  f   95.0  100  100.0   98.33   2    High    100
3    4  sophia  f  100.0  100  100.0  100.00   1    High    100
4    5     amy  f    0.0   45   60.0   31.67  11     Low     55
5    6   james  m   70.0   85   65.0   70.00   8     Low     95
6    7  daniel  m   80.0  100   55.0   75.00   7  Middle    100
7    8   peter  m   50.0   70    0.0   36.67  10     Low     80
8    9   kevin  m  100.0  100   90.0   96.67   3    High    100
9   10    jack  m   90.0  100   70.0   85.00   5  Middle    100
10  11   james  m   70.0   85   65.0   70.00   8     Low     95